In [1]:
import pandas as pd
import csv
import geopandas as gpd
import pandas as pd
import folium
from folium import Choropleth
import json
from IPython.display import display

In [2]:
# Loading all the data
metrpolitan_street = pd.read_csv(r"C:\Users\20223661\Downloads\metropolitan-street-merged.csv")
metrpolitan_street_stopsearch = pd.read_csv(r"C:\Users\20223661\Downloads\metropolitan-stop-and-search-merged.csv")
metrpolitan_outcomes = pd.read_csv(r"C:\Users\20223661\Downloads\metropolitan-outcomes-merged.csv")
filtered_stopsearch = pd.read_csv(r"C:\Users\20223661\Downloads\filtered_stop_and_search.csv")
pas_boroughs = pd.read_csv(r"C:\Users\20223661\Downloads\pas_boroughs.csv")
custody_arrests = pd.read_csv(r"C:\Users\20223661\Downloads\Custody_Arrests_2019_2024.csv", sep=";")
custody_disposals = pd.read_csv(r"C:\Users\20223661\Downloads\Custody_Disposals_2019_2024.csv", sep=";")
pupil_travel_distance = pd.read_csv(r"C:\Users\20223661\Downloads\pupil_home_school_distance.csv", sep=";")

gdf = gpd.read_file(r"C:\Users\20223661\Downloads\london-boroughs_1179.geojson")

In [3]:
# Make columns numerical
pas_boroughs['good_job'] = pd.to_numeric(pas_boroughs['good_job'], errors = 'coerce')
pas_boroughs['trust'] = pd.to_numeric(pas_boroughs['trust'], errors = 'coerce')
pas_boroughs['fair_treatment'] = pd.to_numeric(pas_boroughs['fair_treatment'], errors = 'coerce')
pas_boroughs['dealing_issues'] = pd.to_numeric(pas_boroughs['dealing_issues'], errors = 'coerce')
pupil_travel_distance['far away percentage'] = pupil_travel_distance['far away percentage'].astype(str).str.replace(',', '.').astype(float)

# Drop the unnecessary columns and rows
pas_boroughs = pas_boroughs[['borough', 'trust', 'good_job', 'dealing_issues', 'fair_treatment']]
pupil_travel_distance = pupil_travel_distance[['borough', 'far away percentage']]
pupil_travel_distance = pupil_travel_distance[pupil_travel_distance['borough'] != 'England']

# Calculate average scores for each borough
pas_boroughs_aggregated = pas_boroughs.groupby('borough').mean().reset_index()
pupil_travel_distance_aggregated = pupil_travel_distance.groupby('borough')['far away percentage'].mean().reset_index()

In [8]:
# Make sure borough names match
pupil_travel_distance_aggregated['borough'] = pupil_travel_distance_aggregated['borough'].str.strip()
pas_boroughs_aggregated['borough'] = pas_boroughs_aggregated['borough'].str.strip()

# Merge pas and travel df's
combined_df = pd.merge(pas_boroughs_aggregated, pupil_travel_distance_aggregated, on='borough')

# Merge pas and travel df's with GeoJSON df
merged_pas = gdf.merge(combined_df, left_on='name', right_on='borough', how='left')

In [15]:
def create_map(merged, column, colormap, title):
    m = folium.Map(location=[51.509865, -0.118092], zoom_start=10)
    folium.Choropleth(
        geo_data=merged.__geo_interface__,
        data=merged,
        columns=[merged.index, column],
        key_on='feature.id',
        fill_color=colormap,
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=title,
    ).add_to(m)
    folium.LayerControl().add_to(m)
    return m

# Maps for each metric
good_job_map = create_map(merged_pas, 'good_job', 'YlGn', 'Average Good Job Score')
trust_map = create_map(merged_pas, 'trust', 'PuRd', 'Average Trust Score')
fair_treatment_map = create_map(merged_pas, 'fair_treatment', 'BuPu', 'Average Fair Treatment Score')
dealing_issues_map = create_map(merged_pas, 'dealing_issues', 'OrRd', 'Average Dealing Issues Score')
travel_percentage_map = create_map(merged_pas, 'far away percentage', 'GnBu', 'Percentages of pupils that have to travel far')

# Display maps i
display(good_job_map)
display(trust_map)
display(fair_treatment_map)
display(dealing_issues_map)
display(travel_percentage_map)